In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
import src.config

In [ ]:
import pandas as pd
from deltalake import DeltaTable
from pathlib import Path

In [ ]:
# df = DeltaTable(src.config.SILVER / "account").to_pandas()

In [ ]:
import cdutils.acct_file_creation.core
from datetime import datetime

# # Specific date
specified_date = datetime(2020, 12, 31)
df = cdutils.acct_file_creation.core.query_df_on_date(specified_date)

In [ ]:
df

In [ ]:
df['mjaccttypcd'].unique()

In [ ]:
df = df[df['mjaccttypcd'].isin(['CML','MLN','CNS','MTG','CK','SAV','TD'])].copy()

In [ ]:
# Create Account Type mapping - Easier to understand, based on our major field
def map_account_type(acct_code:str):
    """
    Map mjaccttypcd to friendly Account Type
    """
    mapping = {
        'CML':'Loan',
        'MLN':'Loan',
        'CNS':'Loan',
        'MTG':'Loan',
        'CK':'Deposit',
        'SAV':'Deposit',
        'TD':'Deposit'
    }
    return mapping.get(str(acct_code).upper(), 'Other')

df['Account Type'] = df['mjaccttypcd'].apply(map_account_type)

In [ ]:
df['branchname'].unique()

In [ ]:
df

In [ ]:
region_map = {
    # ——— Attleboro/Taunton ———
    'BCSB - MUNI MAIN OFFICE': 'Attleboro/Taunton',
    'BCSB - MAIN OFFICE': 'Attleboro/Taunton',
    "BCSB - COMM'L LENDING- TAUNTON": 'Attleboro/Taunton',
    'BCSB - MUNI ATTLEBORO BRANCH': 'Attleboro/Taunton',
    'BCSB - DEPOSIT OPERATIONS': 'Attleboro/Taunton',
    'BCSB - NO ATTLEBORO BRANCH': 'Attleboro/Taunton',
    'BRISTOL COUNTY SAVINGS BANK': 'Attleboro/Taunton',
    "BCSB - COMM'L LENDING - ATTLEBORO": 'Attleboro/Taunton',
    'BCSB - BEACON SECURITY CORP': 'Attleboro/Taunton',
    'BCSB - ATTLEBORO BRANCH': 'Attleboro/Taunton',
    'BCSB - MUNI COUNTY STREET BRANCH': 'Attleboro/Taunton',
    'BCSB - REHOBOTH BRANCH': 'Attleboro/Taunton',
    'BCSB - MUNI REHOBOTH BRANCH': 'Attleboro/Taunton',
    'BCSB - MUNI NO ATTLEBORO BRANCH': 'Attleboro/Taunton',
    'BCSB - MUNI RAYNHAM CENTER BRANCH': 'Attleboro/Taunton',
    'BCSB - COUNTY STREET BRANCH': 'Attleboro/Taunton',
    'BCSB - NORTH RAYNHAM BRANCH': 'Attleboro/Taunton',
    'BCSB - RAYNHAM CENTER BRANCH': 'Attleboro/Taunton',
    "BCSB - COMM'L LENDING - FRANKLIN": 'Attleboro/Taunton',
    'BCSB - FRANKLIN BRANCH': 'Attleboro/Taunton',
    'BCSB - CONS INST LENDING- TAUNTON': 'Attleboro/Taunton',
    'BCSB - RESIDENTIAL MTG - ATTLEBORO': 'Attleboro/Taunton',
    'BCSB - RESIDENTIAL MTG- TAUNTON': 'Attleboro/Taunton',
    'BCSB - RESIDENTIAL MTG - FRANKLIN': 'Attleboro/Taunton',
    'BCSB - CONS INST LENDING - ATTLEBORO': 'Attleboro/Taunton',
    'BCSB - SMALL BUSINESS LOAN CENTER': 'Attleboro/Taunton',
    'BCSB - CONTACT CENTER': 'Attleboro/Taunton',
    'BCSB - TAUNTON HIGH SCHOOL': 'Attleboro/Taunton',
    'BCSB - MUNI ATTLEBORO HIGH SCHOOL': 'Attleboro/Taunton',
    'BCSB - ATTLEBORO HIGH SCHOOL': 'Attleboro/Taunton',
    'BCSB - INDIRECT LENDING': 'Attleboro/Taunton',
    'BCSB - WALMART BRANCH': 'Attleboro/Taunton',
    'BCSB - GOVERNMENT FINANCE DEPARTMENT': 'Attleboro/Taunton',

    # ——— South Coast ———
    'BCSB - MUNI FALL RIVER BRANCH': 'South Coast',
    "BCSB - COMM'L LENDING - FALL RIVER": 'South Coast',
    "BCSB - COMM'L LENDING - CANDLEWORKS": 'South Coast',
    "BCSB - COMM'L LENDING - DARTMOUTH": 'South Coast',
    'BCSB - MUNI DARTMOUTH BRANCH': 'South Coast',
    'BCSB - MUNI NB ASHLEY BLVD BRANCH': 'South Coast',
    'BCSB - NB ASHLEY BLVD BRANCH': 'South Coast',
    'BCSB - MUNI CANDLEWORKS BRANCH': 'South Coast',
    'BCSB - MUNI EAST FREETOWN BRANCH': 'South Coast',
    'BCSB - DARTMOUTH BRANCH': 'South Coast',
    'BCSB - EAST FREETOWN BRANCH': 'South Coast',
    'BCSB - FALL RIVER BRANCH': 'South Coast',
    'BCSB - CANDLEWORKS BRANCH': 'South Coast',
    'BCSB - RESIDENTIAL MTG - DARTMOUTH': 'South Coast',
    'BCSB - RESIDENTIAL MTG - FALL RIVER': 'South Coast',
    'BCSB - RESI LENDING - NEW BEDFORD': 'South Coast',
    'BCSB - NB ROCKDALE AVE BRANCH': 'South Coast',

    # ——— Rhode Island ———
    "BCSB - COMM'L LENDING - WARWICK": 'Rhode Island',
    "BCSB - COMM'L LENDING - PROVIDENCE": 'Rhode Island',
    "BCSB - COMM'L LENDING - PAWTUCKET": 'Rhode Island',
    'BCSB - CUMBERLAND': 'Rhode Island',
    'BCSB - PAWTUCKET BRANCH': 'Rhode Island',
    "BCSB - CMM'L LENDING - FNB-RI": 'Rhode Island',
    'BCSB - MUNI PAWTUCKET BRANCH': 'Rhode Island',
    'BCSB - RESIDENTIAL MTG - PAWTUCKET': 'Rhode Island',
    'BCSB - MUNI GREENVILLE': 'Rhode Island',
    'BCSB - GREENVILLE': 'Rhode Island',
    'BCSB - RESI LENDING - WARWICK': 'Rhode Island',
    'BCSB - CONS INST LENDING - PAWTUCKET': 'Rhode Island',
    'BCSB - CONS INST LENDING - FNB-RI': 'Rhode Island',
    'BCSB - MUNI CUMBERLAND': 'Rhode Island',
    'BCSB - RESIDENTIAL MTG - FNB-RI': 'Rhode Island',
    "BCSB - COMM'L LENDING - FNB - RI": 'Rhode Island',
    'BCSB - RESIDENTIAL MTG - FNB - RI': 'Rhode Island',
    'BCSB - CONS INST LENDING - FNB - RI': 'Rhode Island',
    
    
    # ——— Other ———
    'BCSB - RESIDENTIAL MTG - CAPE COD': 'Other',
    # Operational catch-alls (if any are left unmapped in future, they'll fall to 'Other' via the fillna below)
}

In [ ]:
# region_map = {
#     # ——— Attleboro/Taunton ———
#     'BCSB - MUNI MAIN OFFICE': 'Attleboro/Taunton',
#     'BCSB - MAIN OFFICE': 'Attleboro/Taunton',
#     "BCSB - COMM'L LENDING- TAUNTON": 'Attleboro/Taunton',
#     'BCSB - MUNI ATTLEBORO BRANCH': 'Attleboro/Taunton',
#     'BCSB - DEPOSIT OPERATIONS': 'Attleboro/Taunton',
#     'BCSB - NO ATTLEBORO BRANCH': 'Attleboro/Taunton',
#     'BRISTOL COUNTY SAVINGS BANK': 'Attleboro/Taunton',
#     "BCSB - COMM'L LENDING - ATTLEBORO": 'Attleboro/Taunton',
#     'BCSB - BEACON SECURITY CORP': 'Attleboro/Taunton',
#     'BCSB - ATTLEBORO BRANCH': 'Attleboro/Taunton',
#     'BCSB - MUNI COUNTY STREET BRANCH': 'Attleboro/Taunton',
#     'BCSB - REHOBOTH BRANCH': 'Attleboro/Taunton',
#     'BCSB - MUNI REHOBOTH BRANCH': 'Attleboro/Taunton',
#     'BCSB - MUNI NO ATTLEBORO BRANCH': 'Attleboro/Taunton',
#     'BCSB - MUNI RAYNHAM CENTER BRANCH': 'Attleboro/Taunton',
#     'BCSB - COUNTY STREET BRANCH': 'Attleboro/Taunton',
#     'BCSB - NORTH RAYNHAM BRANCH': 'Attleboro/Taunton',
#     'BCSB - RAYNHAM CENTER BRANCH': 'Attleboro/Taunton',
#     "BCSB - COMM'L LENDING - FRANKLIN": 'Attleboro/Taunton',
#     'BCSB - FRANKLIN BRANCH': 'Attleboro/Taunton',
#     'BCSB - CONS INST LENDING- TAUNTON': 'Attleboro/Taunton',
#     'BCSB - RESIDENTIAL MTG - ATTLEBORO': 'Attleboro/Taunton',
#     'BCSB - RESIDENTIAL MTG- TAUNTON': 'Attleboro/Taunton',
#     'BCSB - RESIDENTIAL MTG - FRANKLIN': 'Attleboro/Taunton',
#     'BCSB - CONS INST LENDING - ATTLEBORO': 'Attleboro/Taunton',
#     'BCSB - SMALL BUSINESS LOAN CENTER': 'Attleboro/Taunton',
#     'BCSB - CONTACT CENTER': 'Attleboro/Taunton',
#     'BCSB - TAUNTON HIGH SCHOOL': 'Attleboro/Taunton',
#     'BCSB - MUNI ATTLEBORO HIGH SCHOOL': 'Attleboro/Taunton',
#     'BCSB - ATTLEBORO HIGH SCHOOL': 'Attleboro/Taunton',
#     'BCSB - INDIRECT LENDING': 'Attleboro/Taunton',

#     # ——— South Coast ———
#     'BCSB - MUNI FALL RIVER BRANCH': 'South Coast',
#     "BCSB - COMM'L LENDING - FALL RIVER": 'South Coast',
#     "BCSB - COMM'L LENDING - CANDLEWORKS": 'South Coast',
#     "BCSB - COMM'L LENDING - DARTMOUTH": 'South Coast',
#     'BCSB - MUNI DARTMOUTH BRANCH': 'South Coast',
#     'BCSB - MUNI NB ASHLEY BLVD BRANCH': 'South Coast',
#     'BCSB - NB ASHLEY BLVD BRANCH': 'South Coast',
#     'BCSB - MUNI CANDLEWORKS BRANCH': 'South Coast',
#     'BCSB - MUNI EAST FREETOWN BRANCH': 'South Coast',
#     'BCSB - DARTMOUTH BRANCH': 'South Coast',
#     'BCSB - EAST FREETOWN BRANCH': 'South Coast',
#     'BCSB - FALL RIVER BRANCH': 'South Coast',
#     'BCSB - CANDLEWORKS BRANCH': 'South Coast',
#     'BCSB - RESIDENTIAL MTG - DARTMOUTH': 'South Coast',
#     'BCSB - RESIDENTIAL MTG - FALL RIVER': 'South Coast',
#     'BCSB - RESI LENDING - NEW BEDFORD': 'South Coast',

#     # ——— Rhode Island ———
#     "BCSB - COMM'L LENDING - WARWICK": 'Rhode Island',
#     "BCSB - COMM'L LENDING - PROVIDENCE": 'Rhode Island',
#     "BCSB - COMM'L LENDING - PAWTUCKET": 'Rhode Island',
#     'BCSB - CUMBERLAND': 'Rhode Island',
#     'BCSB - PAWTUCKET BRANCH': 'Rhode Island',
#     "BCSB - CMM'L LENDING - FNB-RI": 'Rhode Island',
#     'BCSB - MUNI PAWTUCKET BRANCH': 'Rhode Island',
#     'BCSB - RESIDENTIAL MTG - PAWTUCKET': 'Rhode Island',
#     'BCSB - MUNI GREENVILLE': 'Rhode Island',
#     'BCSB - GREENVILLE': 'Rhode Island',
#     'BCSB - RESI LENDING - WARWICK': 'Rhode Island',
#     'BCSB - CONS INST LENDING - PAWTUCKET': 'Rhode Island',
#     'BCSB - CONS INST LENDING - FNB-RI': 'Rhode Island',
#     'BCSB - MUNI CUMBERLAND': 'Rhode Island',
#     'BCSB - RESIDENTIAL MTG - FNB-RI': 'Rhode Island',

#     # ——— Other ———
#     'BCSB - RESIDENTIAL MTG - CAPE COD': 'Other',
#     # Operational catch-alls (if any are left unmapped in future, they'll fall to 'Other' via the fillna below)
# }

# # Create the Region column from the mapping
# import numpy as np
# df['Region'] = df['branchname'].map(region_map).fillna(
#     np.where(df['branchname'].str.contains(r'Warwick|Providence|Pawtucket|Cumberland|Greenville|FNB-RI', case=False), 'Rhode Island',
#     np.where(df['branchname'].str.contains(r'Fall River|Dartmouth|East Freetown|New Bedford|Candleworks|Ashley Blvd', case=False), 'South Coast',
#     np.where(df['branchname'].str.contains(r'Attleboro|Franklin|Raynham|Taunton|Rehoboth|County Street|Main Office', case=False), 'Attleboro/Taunton', 'Other'))))



In [ ]:
# --- Normalize, Map Regions, and Emit "Other" Review --------------------------
import re
import pandas as pd

# 1) Normalize BRANCH names in a new column (keep original for reference)
def normalize_branch(series: pd.Series) -> pd.Series:
    s = series.fillna("").astype(str).str.upper()

    # Unify quotes/apostrophes and whitespace/hyphens
    s = s.str.replace(r"[‘’ʼ´`]", "'", regex=True)           # curly -> straight
    s = s.str.replace(r"\s*-\s*", " - ", regex=True)         # spaces around hyphen
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()   # collapse spaces

    # Fix a couple of known variants/typos seen historically
    s = s.str.replace("CMM'L", "COMM'L", regex=False)
    s = s.str.replace("COMM’L", "COMM'L", regex=False)

    return s

df["branch_std"] = normalize_branch(df["branchname"])

# 2) Ensure the region map uses UPPERCASE keys (safe even if yours already are)
region_map_upper = {str(k).upper(): v for k, v in region_map.items()}

# 3) Exact match mapping first
df["Region"] = df["branch_std"].map(region_map_upper)

# 4) Regex fallback by geography (safety net; exact map above is the source of truth)
_fallback_patterns = [
    (r"\b(WARWICK|PROVIDENCE|PAWTUCKET|CUMBERLAND|GREENVILLE|FNB-RI)\b", "Rhode Island"),
    (r"\b(FALL RIVER|DARTMOUTH|EAST FREETOWN|NEW BEDFORD|CANDLEWORKS|ASHLEY BLVD)\b", "South Coast"),
    (r"\b(ATTLEBORO|FRANKLIN|RAYNHAM|TAUNTON|REHOBOTH|COUNTY STREET|MAIN OFFICE)\b", "Attleboro/Taunton"),
]

def fallback_region(name: str) -> str | None:
    for pat, region in _fallback_patterns:
        if re.search(pat, name):
            return region
    return None

df["Region"] = df["Region"].fillna(df["branch_std"].apply(lambda x: fallback_region(x) or "Other"))

# 5) Optional: fix Region order so it’s stable across runs
REGION_ORDER = ["Attleboro/Taunton", "South Coast", "Rhode Island", "Other"]
df["Region"] = pd.Categorical(df["Region"], categories=REGION_ORDER, ordered=True)

# 6) Your aggregation (unchanged)
grouped_df = (
    df.groupby(["Region", "Account Type"], observed=True)
      .agg(NetBalance_sum=("Net Balance", "sum"))
      .reset_index()
)

# 7) Always emit a review table for unmapped/“Other”
other_base = df[df["Region"] == "Other"].copy()

# One row per normalized branch, with examples of original values and totals
examples = (
    other_base.groupby("branch_std")["branchname"]
    .apply(lambda x: sorted(set(x))[:3])  # up to 3 example raw names
    .reset_index(name="examples")
)

other_df = (
    other_base.groupby("branch_std", as_index=False)
    .agg(
        n_accounts=("branch_std", "size"),
        NetBalance_sum=("Net Balance", "sum"),
    )
    .merge(examples, on="branch_std", how="left")
    .sort_values(["n_accounts", "NetBalance_sum"], ascending=[False, False])
)

# 8) (Nice to have) quick coverage stats for your log
mapped_rate = (df["Region"] != "Other").mean()
print(f"Region mapping coverage: {mapped_rate:.1%} of rows; {other_df.shape[0]} unmapped branch_std values.")
# -----------------------------------------------------------------------------



In [ ]:
other_df

In [ ]:
# Performed 1 aggregation grouped on columns: 'Region', 'Account Type'
grouped_df = df.groupby(['Region', 'Account Type']).agg(NetBalance_sum=('Net Balance', 'sum')).reset_index()

In [ ]:
grouped_df